In [ ]:
from mongo_episode import Episode
from mongo_auteur import Auteur, AuthorChecker
import pandas as pd


def ajoute_auteurs(episode: Episode, verbose=False):
    affiche_episode = f"""
    Date: {episode.date.strftime("%d %b %Y")}
    Titre: {episode.titre}
    Description: {episode.description}

    """
    print(affiche_episode)

    auteurs = episode.get_all_auteurs()
    print(f"La liste des auteurs : {auteurs}\n")

    ac = AuthorChecker(episode)
    auteur_traitement_df = pd.DataFrame(
        columns=["auteur_corrige", "detection", "existait_en_base", "anomalie"]
    )
    analyse_dict = {}

    for auteur in auteurs:
        auteur_corrige_dict = ac.check_author(
            auteur, return_details=True, verbose=verbose
        )
        if verbose:
            print(auteur_corrige_dict)
        # check_author retourne None dans author_corrected si le process via rss:metadata, db, llm, web search n'a rien renvoye
        # c'est donc une anomalie
        anomalie = True if auteur_corrige_dict["author_corrected"] is None else False
        # dans auteur_corrige c'est le nom de l'auteur si il est different de l'original sinon None
        auteur_corrige = (
            auteur_corrige_dict["author_corrected"]
            if auteur_corrige_dict["author_corrected"]
            != auteur_corrige_dict["author_original"]
            else None
        )
        nom_auteur = auteur_corrige_dict["author_corrected"] if not None else auteur
        # a quel endroit a ete detecte l'auteur (rss:metadata, db, llm, web search)
        detection = auteur_corrige_dict["source"]
        existait_en_base = False if anomalie else Auteur(nom_auteur).exists()
        auteur_traitement_df.loc[auteur] = [
            auteur_corrige,
            detection,
            existait_en_base,
            anomalie,
        ]
        if not anomalie:
            aut = Auteur(nom_auteur)
            aut.keep()
        else:
            analyse_dict[auteur] = {
                "analyse": auteur_corrige_dict["analyse"],
                "score": auteur_corrige_dict["score"],
            }

    print(episode.date)
    print(auteur_traitement_df)
    if len(analyse_dict) > 0:
        print("\nAnalyse des anomalies\n")
    for auteur, details in analyse_dict.items():
        print(f"{auteur} -> {details['analyse']} (score: {details['score']})")

In [ ]:
import datetime

date = datetime.datetime(2021, 1, 17)

episode = Episode.from_date(date=date)
assert episode is not None, "no episode for date {}".format(date)

ajoute_auteurs(episode, verbose=True)


    Date: 17 Jan 2021
    Titre: Livres : que valent les derniers de Judith Perrignon, Mathieu Lindon, Louise Erdrich, Ivan Jablonka… ?
    Description: Ces
 livres vont-ils faire partie de votre bibliothèque : "Là où nous 
dansions" de Judith Perrignon, "Hervelino" de Mathieu Lindon, "L’enfant 
de la prochaine aurore" de Louise Erdrich, "L’ami arménien" d'Andréï 
Makine, "Un garçon comme vous et moi" de Ivan Jablonka ?

    
La liste des auteurs : ['Judith Pérignon', 'Mathieu Lindon', 'Hervé Guibert', 'André Mackin', 'Louise Erdrich', 'Yvan Jablonka', 'Hervé Letellier', 'Nelly Capriélian', 'Arnaud Dudek', 'Paul Valéry', 'Olivia de Lamberterie', 'Michel Foucault', 'Jules Verne', 'Mario Kaplan', 'Michel Crépu', 'Antoine Duban', 'Sylvain Vergès', 'Isabelle Schippers', 'Ahmed Boussetla', 'Michel Trépu', 'Jean-Claude Raspien-Jas de La Croix', 'Arnaud Vivian']

Trouvé avec rss:metadata: Judith Perrignon
{'author_original': 'Judith Pérignon', 'author_corrected': 'Judith Perrignon', 'source'